In [1]:
#pip install gazpacho

In [2]:
from gazpacho import get, Soup
import numpy
import pandas
import requests, bs4
import re, os
import time

In [3]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://wyeastlab.com/product/german-ale/'

# fetch step
html = get(url)

# extract step
soup = Soup(html)

In [4]:
# use find() to parse a specific HTML
gaz_data = soup.find('div', {'class':'container'})
gaz_data

[<div class="grid-container">
 
         <div class="grid-x grid-margin-x">
 
             <div class="cell small-12">
 
                 <ul class="accordion-menu vertical menu" data-accordion-menu="None" data-multi-open="false" data-submenu-toggle="true">
 
                     <li class="sub-menu active menu-item menu-item-has-children menu-our-products"><a href="https://wyeastlab.com/our-products/">Our Products</a>
 <ul class="menu">
 	<li class="menu-item menu-yeast-cultures"><a href="https://wyeastlab.com/yeast-cultures/">Yeast & Cultures</a></li>
 	<li class="menu-item menu-supporting-fermentation-products"><a href="https://wyeastlab.com/supporting-fermentation-products/">Supporting Fermentation Products</a></li>
 	<li class="menu-item menu-activator-smack-pack-system"><a href="https://wyeastlab.com/activator-smack-pack-system/">Activator Smack-Pack System™</a></li>
 	<li class="menu-item menu-private-culture-collection"><a href="https://wyeastlab.com/private-culture-collection/

### Pull The Strain Name

In [5]:
# Pull The Strain Name

def pull_strain_name(soup):
    strain_name = soup.find('h2', {'class':'headline'})
    x = str(strain_name)
    start = x.find('Strain: ')
    end = x.find('</h2>')
    strain = x[start:end]
    # print(strain)
    return strain

In [6]:
pull_strain_name(soup)

'Strain: 1007'

### Pull The Strain Number

In [7]:
# Pull The Strain Number

def pull_strain_number(soup):
    strain_number = soup.find('h3', {'class':'subheadline'})
    x = str(strain_number)
    start = x.find('>')
    end = x.find('</h3>')
    number = x[start+1:end]
    # print(number)
    return number

In [8]:
pull_strain_number(soup)

'German Ale'

### Pull The Description

In [9]:
# Pull and save the description

def pull_strain_description(soup):
    description = soup.find('div', {'class':'description'})
    #print(description)
    x= str(description)
    start = x.find('Profile:')
    start = start + 20
    end = x.find('gform_fields_2')
    end = end - 23
    description = x[start:end]
    # print(description)
    return description

In [10]:
pull_strain_description(soup)

'    A true top cropping yeast with low ester formation and a broad temperature range. Fermentation at higher temperatures may produce mild fruitiness. This powdery strain results in yeast that remains in suspension post fermentation. Beers mature rapidly, even when cold fermentation is used. Low or no detectable diacetyl.\n '

### Get Stats on Strain

In [11]:
def pull_stats(soup):
    raw = soup.find('ul', {'class':'features'})
    x= str(raw)
    print(x)
    fs = x.find('<h6>Flocculation</h6>') + 32
    fe = x.find('<h6>Flocculation</h6>') + 39
    flocculation = x[fs:fe]
    # print(flocculation)

    attens = x.find('<h6>Apparent Attenuation</h6>') + 40
    attene = x.find('<h6>Apparent Attenuation</h6>') + 50
    attenuation = x[attens:attene]
    # print(attenuation)

    abve = x.find('<h6>Apparent ABV Tolerance</h6>') + 42
    abvs = x.find('<h6>Apparent ABV Tolerance</h6>') + 45
    abv = x[abve:abvs]
    # print(abv)

    temps = x.find('<h6>Temperature Range</h6>') + 37
    tempe = x.find('<h6>Temperature Range</h6>') + 54
    temp = x[temps:tempe]
    # print(temp)

    return(flocculation, attenuation, abv, temp)

In [12]:
pull_stats(soup)

<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>73 - 77%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>11%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>55-68°F (13-20°C)</span>
  </li>
</ul>


('Low</sp', '73 - 77%</', '11%', '55-68°F (13-20°C)')

### Pull all the urls

In [13]:
main_url = 'https://wyeastlab.com/yeast-cultures/'

In [14]:
url_ends = ['german-ale', 'american-wheat', 'british-cask-ale', 'london-ale', 'american-ale', 'irish-ale', 'wyeast-bohemian-ale-blend',
           'british-ale', 'whitbread-ale', 'ringwood-ale', 'burton-ipa-blend', 'belgian-abbey-style-ale', 'west-coast-ipa', 'american-ale-ii',
           'thames-valley-ale', 'london-ale-iii', 'northwest-ale', 'british-ale-ii', 'belgian-strong-ale', 'dennys-favorite-50', 'west-yorkshire-ale',
           'belgian-stout', 'scottish-ale', 'belgian-abbey-style-ale-ii', 'english-special-bitter', 'thames-valley-ale-ii', 'london-esb-ale', 'budvar-lager',
           'pilsner-urquell-h-strain', 'gambrinus-style-lager', 'pilsen-lager', 'american-lager', 'danish-lager', 'rocky-mountain-lager',
           'california-lager', 'bohemian-lager', 'bavarian-lager', 'european-lager', 'north-american-lager', 'czech-pils', 'munich-lager', 'munich-lager-ii',
           'hella-bock-lager', 'klsch', 'klsch-ii', 'octoberfest-lager-blend', 'staro-prague-lager', 'saison-brett-blend', 'bavarian-wheat-blend','weihenstephan-weizen',
           'german-wheat','forbidden-fruit','belgian-ardennes','bavarian-wheat','belgian-schelde-ale','french-saison','belgian-saison','bire-de-garde','farmhouse-ale',
           'flanders-golden-ale','belgian-high-gravity','belgian-dark-ale','canadianbelgian-ale','belgian-wheat','belgian-witbier', 'brettanomyces-bruxellensis', 'brettanomyces-claussenii',
            'lactobacillus-brevis', 'lactobacillus-buchneri', 'brettanomyces-lambicus', 'pediococcus-damnosus', 'old-ale-blend']

In [15]:
url_front = 'https://wyeastlab.com/product/'

In [16]:
full_url = []

for i in url_ends:
    x = url_front + i
    full_url.append(x)
    print(x)

https://wyeastlab.com/product/german-ale
https://wyeastlab.com/product/american-wheat
https://wyeastlab.com/product/british-cask-ale
https://wyeastlab.com/product/london-ale
https://wyeastlab.com/product/american-ale
https://wyeastlab.com/product/irish-ale
https://wyeastlab.com/product/wyeast-bohemian-ale-blend
https://wyeastlab.com/product/british-ale
https://wyeastlab.com/product/whitbread-ale
https://wyeastlab.com/product/ringwood-ale
https://wyeastlab.com/product/burton-ipa-blend
https://wyeastlab.com/product/belgian-abbey-style-ale
https://wyeastlab.com/product/west-coast-ipa
https://wyeastlab.com/product/american-ale-ii
https://wyeastlab.com/product/thames-valley-ale
https://wyeastlab.com/product/london-ale-iii
https://wyeastlab.com/product/northwest-ale
https://wyeastlab.com/product/british-ale-ii
https://wyeastlab.com/product/belgian-strong-ale
https://wyeastlab.com/product/dennys-favorite-50
https://wyeastlab.com/product/west-yorkshire-ale
https://wyeastlab.com/product/belgian

In [17]:
import pandas as pd

def loadTable(soup):
    strain = pull_strain_name(soup)
    #pullMetrics(soup)
    number = pull_strain_number(soup)
    flocculation, attenuation, abv, temp = pull_stats(soup)
    description = pull_strain_description(soup)
    entry = {'Strain': [strain],
             'Number' : [number],
        'Attenuation' : [attenuation],
        'Flocculation' : [flocculation],
        'Alcohol_Tolerance' : [abv],
        'Fermentation_Temperature' : [temp],
        'Description' : [description]}
    df = pd.DataFrame(entry)
    return df

In [18]:
loadTable(soup)

<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>73 - 77%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>11%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>55-68°F (13-20°C)</span>
  </li>
</ul>


,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description
0,Strain: 1007,German Ale,73 - 77%</,Low</sp,11%,55-68°F (13-20°C),A true top cropping yeast with low ester f...


In [19]:
df = loadTable(soup)
df1 = df
df1

<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>73 - 77%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>11%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>55-68°F (13-20°C)</span>
  </li>
</ul>


,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description
0,Strain: 1007,German Ale,73 - 77%</,Low</sp,11%,55-68°F (13-20°C),A true top cropping yeast with low ester f...


In [20]:
for i in full_url:
  print(i)
  # fetch step
  html = get(i)

  # extract step
  soup = Soup(html)
  df = loadTable(soup)
  df1 = pd.concat([df1, df])

https://wyeastlab.com/product/german-ale
<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>73 - 77%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>11%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>55-68°F (13-20°C)</span>
  </li>
</ul>
https://wyeastlab.com/product/american-wheat
<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>74 - 78%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>10%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>58-74°F (14-23°C)</span>
  </li>
</ul>
https://wyeastlab.com/product/british-cask-ale
<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Medium High</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>74 - 77%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>9%</span>
 

<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Medium</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>70 - 85%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>12%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>65-75°F (18-24°C)</span>
  </li>
</ul>
https://wyeastlab.com/product/scottish-ale
[<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>High</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>69 - 73%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>12%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>55-75°F (13-24°C)</span>
  </li>
</ul>, <ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>High</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>69 - 73%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>12%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>55-75°F (13-24°C)</span>
  </li>
</

<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>73 - 77%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>10%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>56-70°F (13-21°C)</span>
  </li>
</ul>
https://wyeastlab.com/product/klsch-ii
<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>73 - 77%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>10%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>55-70°F (13-21°C)</span>
  </li>
</ul>
https://wyeastlab.com/product/octoberfest-lager-blend
<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low Medium</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>73 - 77%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>9%</span>
  </li>
  <li>
    <h6>Temperature Range</

<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Medium</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>80%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>12%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>60-75°F (16-24°C)</span>
  </li>
</ul>
https://wyeastlab.com/product/lactobacillus-brevis
<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low - High</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>70 - 90%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>9%</span>
  </li>
  <li>
    <h6>Temperature Range</h6>
    <span>60-95°F (16-35°C)</span>
  </li>
</ul>
https://wyeastlab.com/product/lactobacillus-buchneri
<ul class="features">
  <li>
    <h6>Flocculation</h6>
    <span>Low - High</span>
  </li>
  <li>
    <h6>Apparent Attenuation</h6>
    <span>70 - 90%</span>
  </li>
  <li>
    <h6>Apparent ABV Tolerance</h6>
    <span>9%</span>
  </li>
  <li>
    <h6>Temp

In [21]:
df1

,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,Description
0,Strain: 1007,German Ale,73 - 77%</,Low</sp,11%,55-68°F (13-20°C),A true top cropping yeast with low ester f...
0,Strain: 1007,German Ale,73 - 77%</,Low</sp,11%,55-68°F (13-20°C),A true top cropping yeast with low ester f...
0,Strain: 1010,American Wheat,74 - 78%</,Low</sp,10%,58-74°F (14-23°C),"A strong fermenting, true top cropping yea..."
0,Strain: 1026-PC,British Cask Ale,74 - 77%</,Medium,9%<,63-72°F (17-22°C),A great yeast choice for any cask-conditio...
0,Strain: 1028,London Ale,73 - 77%</,Low Med,11%,60-72°F (16-22°C),A rich mineral profile that is bold and cr...
...,...,...,...,...,...,...,...
0,Strain: 5223-PC,Lactobacillus brevis,70 - 90%</,Low - H,9%<,60-95°F (16-35°C),Unlike most lactic acid bacteria used in b...
0,Strain: 5335,Lactobacillus buchneri,70 - 90%</,Low - H,9%<,60-95°F (16-35°C),This culture produces moderate levels of a...
0,Strain: 5526,Brettanomyces lambicus,78 - 84%</,Medium<,12%,60-75°F (16-24°C),This is a wild yeast strain isolated from ...
0,Strain: 5733,Pediococcus damnosus,70 - 90%</,Low - H,9%<,60-95°F (16-35°C),Lactic acid bacteria used in the productio...


# Pulling all the urls that are needed

In [23]:
df1.to_csv('Wyeast_Strains.csv')